In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

title = 'woplus'
path = '../../sources/data'

In [6]:
df_net = pd.read_table('{}/dataset_2015_filter_net.txt'.format(path), sep='|')
df_net_piv = df_net.pivot(index='imsi', columns='mon', values='nettype')
df_net_piv.columns = ['net01','net02','net03','net04','net05','net06','net07','net08','net09','net10','net11','net12']
ictype = {}
ictype['catbin'] = ['net01','net02','net03','net04','net05','net06','net07','net08','net09','net10','net11','net12']

In [9]:
def Cat_to_bin(das, a = 0.01):
    '''Transfrom a categorical column to onehotencoding'''
    tmp = pd.value_counts(das)/das.shape[0]
    cat = list(tmp.index[tmp > a])
    enc = OneHotEncoder(n_values = len(cat)+1, sparse = False)
    xbin = enc.fit_transform(np.transpose(
            [das.astype("category").cat.set_categories(cat).cat.rename_categories(1+np.arange(len(cat))).astype("float").fillna(0).values]))[:,1:]     
    dabin = pd.DataFrame(xbin, columns = ["{}_{}".format(das.name, x) for x in cat], index = das.index)    # origin
#     dabin = pd.DataFrame(xbin, columns = ["{}_{}".format(das.name, x) for x in (1+np.arange(len(cat)))], index = das.index) 
    if(tmp[tmp <= a].sum() > a):
        dabin = pd.concat([dabin, pd.DataFrame({"{}_Others".format(das.name):das.notnull()-dabin.sum(axis = 1)})], axis = 1)
    if(dabin.shape[1] == 2):
        dabin = pd.DataFrame({das.name: xbin[:,0]}, index = das.index)
    return(dabin)

In [11]:
Cat_to_bin(df_net_piv.loc[:, ictype['catbin'][0]]).head()

,mon01
imsi,
00002ec9f6870677ca801c213ce87c02,0
0000be0fbdbe7f8ff47f7ec3e077e489,1
000166f550781551e1eb11cfcd3f4267,1
0001a48e7fca2e7e3279a5f275525cdb,1
0001b4ebaa72f0e4c2e79a0791bf3002,1


In [13]:
dacatbin = pd.concat(map(lambda i: Cat_to_bin(df_net_piv.loc[:,i], a = 0.01), ictype['catbin']), axis = 1)
dacatbin.head()

,mon01,mon02,mon03,mon04,mon05,mon06,mon07,mon08,mon09,mon10,mon11,mon12
imsi,,,,,,,,,,,,
00002ec9f6870677ca801c213ce87c02,0,0,0,0,0,0,0,0,0,0,0,0
0000be0fbdbe7f8ff47f7ec3e077e489,1,1,1,1,1,1,1,1,1,1,1,1
000166f550781551e1eb11cfcd3f4267,1,1,1,1,1,1,1,1,1,1,1,1
0001a48e7fca2e7e3279a5f275525cdb,1,1,1,1,1,1,1,1,1,1,1,1
0001b4ebaa72f0e4c2e79a0791bf3002,1,1,1,1,1,1,1,1,1,1,1,1
